In [ ]:
from google.colab import files
f = files.upload()

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1dh6w4ng
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-1dh6w4ng
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 645kB 15.5MB/s 
  Created wheel for transformers: filename=transformers-4.9.0.dev0-cp37-none-any.whl size=2584040 sha256=0c77119bc2279dfab642bfcce6fe69ebef144a088457bd98f41dca81dd3ec960
  Stored in directory: /tmp/pip-ephem-wheel-cache-nzyq2i0b/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: transformers 4.8.2
    Uninstalling transformers-4.8.2:
      Successfully uninstalled transformers-4.8.2


In [ ]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 78437, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 78437 (delta 42), reused 85 (delta 34), pack-reused 78335
Receiving objects: 100% (78437/78437), 61.10 MiB | 20.27 MiB/s, done.
Resolving deltas: 100% (56033/56033), done.


In [ ]:
#!pip install --upgrade -q transformers tokenizers
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import pipeline
from tokenizers import Tokenizer
from tqdm.auto import tqdm
import torch
import random
import numpy as np
import re
if torch.cuda.is_available():
  device = torch.device("cuda")
  print(f'There are {torch.cuda.device_count()} GPU(s) available.')
  print('Device name:', torch.cuda.get_device_name(0))
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla V100-SXM2-16GB


In [ ]:
df = pd.read_csv('/content/mentalillness.csv')
df = df['0']
df = df.astype(str).apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df = df.replace('\n','', regex=True)
df = df.replace('nan','', regex=True)
df.head()
#df = df.set_index('label')
#df = df.drop([1,2,3,4,5])

#df.reset_index(drop=True, inplace=True)

0    I feel this way most days and I'm getting sick...
1             Saw this today and wanted to share it.  
2    I can understand if my situation was a permane...
3    So suicidal thoughts are really common to me b...
4    If any wants to share their experience.I read ...
Name: 0, dtype: object

In [ ]:
df = df.dropna(axis=0, how='any')

In [ ]:
df.isna().sum()

0

In [ ]:
temp = []
for line in df:
  web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
  line = web_address.sub('', line)
  temp.append(line)

df = temp

df = pd.DataFrame(df)

In [ ]:
df.head()

,0
0,I feel this way most days and I'm getting sick...
1,Saw this today and wanted to share it.
2,I can understand if my situation was a permane...
3,So suicidal thoughts are really common to me b...
4,If any wants to share their experience.I read ...


In [ ]:
train, test = train_test_split(
    df,
    test_size = 0.1,
    shuffle=True
)

In [ ]:
test[0]

7749     I have schizoaffective bipolar disorder. And I...
19493    Ive recently started therapy after waiting for...
2940     Early Life Stress Can Cause Adult PTSD, Even W...
5048     please don't reply by telling me i don't becau...
20972    Hi,  I'm really scared to write here, I'm real...
                               ...                        
23131    I am bipolar type I with mixed features. Ive b...
10975    Anyone else getting FUCKING SICK of working fo...
14890    I seem like the bad guy now because people thi...
24444                                                     
15242    I hope this Fourth of July is bearable for you...
Name: 0, Length: 2775, dtype: object

In [ ]:
with open('train.txt', 'w') as f:
  for line in train[0]:
    line = '<|startoftext|>' + line + '<|endoftext|>'
    f.write(line + '\n')
  f.close()

with open('test.txt', 'w') as f:
  for line in test[0]:
    line = '<|startoftext|>' + line + '<|endoftext|>'
    f.write(line + '\n')
  f.close()

In [ ]:
import os
os.chdir("/content/transformers/examples/pytorch/")
os.chdir("./language-modeling")

!pip install -r requirements.txt

!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "/content/train.txt" \
--do_train \
--validation_file "/content/test.txt" \
--do_eval \
--per_device_train_batch_size 1 \
--num_train_epochs 25 \
--save_steps 15000 \
--fp16 \
--output_dir="/content/finetuned" \
--overwrite_output_dir

files.download("/content/finetuned")

2021-07-18 07:57:33.859896: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
07/18/2021 07:57:35 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/18/2021 07:57:35 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_colu

In [ ]:
model = GPT2LMHeadModel.from_pretrained('/content/finetuned')
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
input_sequence = "Madam, do pardon the interruption but I must ask you to removeth thine spatula from betwixt my buttocks"

input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

generated_text_samples = model.generate(
    input_ids, 
    max_length=50000,
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=False
)


with open('generated.txt', 'w') as f:
  f.write("Output:\n" + 50 * '-' + '\n')
  for i, sample_output in enumerate(generated_text_samples):
    f.write("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)) + '\n')




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
